# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [35]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [36]:
# your code here
orders = pd.read_csv('Orders.zip')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [37]:
# your code here
# Sub Problem 1: How to aggregate the amount_spent for unique customers?
customer_total = orders.groupby('CustomerID').agg({'amount_spent':'sum'})
customer_total.head()

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40


In [38]:
# Sub Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?
vip_threshold = customer_total['amount_spent'].quantile(0.95)
preferred_threshold = customer_total['amount_spent'].quantile(0.75)
#verifying
vip_threshold, preferred_threshold

(5840.181999999982, 1661.64)

In [39]:
vip_customers = customer_total[customer_total['amount_spent'] > vip_threshold]
preferred_customers = customer_total[(customer_total['amount_spent'] > preferred_threshold) & (customer_total['amount_spent'] <= vip_threshold)]
#verifying
display(vip_customers.head())
display(preferred_customers.head())

,amount_spent
CustomerID,
12346,77183.60
12357,6207.67
12359,6372.58
12409,11072.67
12415,124914.53


,amount_spent
CustomerID,
12347,4310.00
12348,1797.24
12349,1757.55
12352,2506.04
12356,2811.43


In [40]:
# Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?
customer_total['customer_label'] = 'Normal'
customer_total.loc[customer_total.index.isin(vip_customers.index), 'customer_label'] = 'VIP'
customer_total.loc[customer_total.index.isin(preferred_customers.index), 'customer_label'] = 'Preferred'
#verifying
customer_total.head()

,amount_spent,customer_label
CustomerID,,
12346,77183.60,VIP
12347,4310.00,Preferred
12348,1797.24,Preferred
12349,1757.55,Preferred
12350,334.40,Normal


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [50]:
# Filter the DataFrame to include only VIP customers
vip_customers_df = orders[orders['CustomerID'].isin(vip_customers.index)]
vip_customers_df.tail()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
397883,541868,581584,85038,2011,12,5,12,6 chocolate love heart t-lights,48,2011-12-09 12:25:00,1.85,13777,United Kingdom,88.8
397905,541890,581586,22061,2011,12,5,12,large cake stand hanging strawbery,8,2011-12-09 12:49:00,2.95,13113,United Kingdom,23.6
397906,541891,581586,23275,2011,12,5,12,set of 3 hanging owls ollie beak,24,2011-12-09 12:49:00,1.25,13113,United Kingdom,30.0
397907,541892,581586,21217,2011,12,5,12,red retrospot round cake tins,24,2011-12-09 12:49:00,8.95,13113,United Kingdom,214.8
397908,541893,581586,20685,2011,12,5,12,doormat red retrospot,10,2011-12-09 12:49:00,7.08,13113,United Kingdom,70.8


In [58]:
# Count unique VIP customers by country
vip_customers_by_country = vip_customers_df.groupby('Country')['CustomerID'].nunique()
vip_customers_by_country.head(20)

,CustomerID
Country,
Australia,2
Belgium,1
Channel Islands,1
Cyprus,1
Denmark,1
EIRE,2
Finland,1
France,9
Germany,10


In [59]:
# A country with the most VIPs:
print(f'A country with the most VIP customers is: {vip_customers_by_country.idxmax()} with {vip_customers_by_country.max()} customers')

A country with the most VIP customers is: United Kingdom with 177 customers


## Q3: How to identify which country has the most VIP+Preferred Customers

*   List item
*   List item

combined?

In [62]:
# Filter the DataFrame to include only Preferred customers
preferred_customers_df = orders[orders['CustomerID'].isin(preferred_customers.index)]
preferred_customers_df

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397900,541885,581585,21684,2011,12,5,12,small medina stamped metal bowl,12,2011-12-09 12:31:00,0.85,15804,United Kingdom,10.20
397901,541886,581585,22398,2011,12,5,12,magnets pack of 4 swallows,12,2011-12-09 12:31:00,0.39,15804,United Kingdom,4.68
397902,541887,581585,23328,2011,12,5,12,set 6 school milk bottles in crate,4,2011-12-09 12:31:00,3.75,15804,United Kingdom,15.00
397903,541888,581585,23145,2011,12,5,12,zinc t-light holder star large,12,2011-12-09 12:31:00,0.95,15804,United Kingdom,11.40


In [63]:
# Count unique Preferred customers by country
preferred_customers_by_country = preferred_customers_df.groupby('Country')['CustomerID'].nunique()

# Print Preferred customer counts by country
preferred_customers_by_country

,CustomerID
Country,
Australia,2
Austria,3
Belgium,11
Canada,1
Channel Islands,3
Cyprus,3
Denmark,2
EIRE,1
Finland,4


In [64]:
# Identify and print the country with the most Preferred customers
print(f'A country with the most preferred customers is: {preferred_customers_by_country.idxmax()} with {preferred_customers_by_country.max()} customers')

A country with the most preferred customers is: United Kingdom with 755 customers


In [65]:
#seeing that both VIP and preferred customers are mostly located in United Kingdom we can jsut suь up the values
most_vip_and_preferred_customers = vip_customers_by_country.max() + preferred_customers_by_country.max()
print(f'The total number of most VIP and preferred customers is: {most_vip_and_preferred_customers}')

The total number of most VIP and preferred customers is: 932


In [66]:
#checking the VIPs & Preferred and sum em up
# Extract counts for the United Kingdom
uk_vip_count = vip_customers_by_country.get('United Kingdom', 0)
uk_preferred_count = preferred_customers_by_country.get('United Kingdom', 0)

# Calculate the total for the United Kingdom
total_uk_customers = uk_vip_count + uk_preferred_count

print(f'The total number of VIP and Preferred customers in the United Kingdom is: {total_uk_customers}')

#seems correct, 932 = 932

The total number of VIP and Preferred customers in the United Kingdom is: 932
